# Keras exercise

In this exercise you will be creating a Keras model by loading a data set, preprocessing input data, building a Sequential Keras model and compiling the model with a training configuration. Afterwards, you train your model on the training data and evaluate it on the test set. To finish this exercise, you will past the accuracy of your model to the Coursera grader.

This notebook is tested in IBM Watson Studio under python 3.6

## Data

For this exercise we will use the Reuters newswire dataset. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes, just as in the IMDB data we encountered in lecture 5 of this series. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

## Goal

We want to create a Multi-layer perceptron (MLP) using Keras which we can train to classify news items into the specified 46 topics.

## Instructions

We start by installing and importing everything we need for this exercise:

In [1]:
!pip install tensorflow==2.2.0rc0

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200328204747-0000
KERNEL_ID = 1ca974b9-d54a-4ede-9f1d-6c83bb6be4e0
    100% |################################| 515.9MB 45kB/s  eta 0:00:01s eta 0:00:09    11% |###                             | 60.9MB 61.5MB/s eta 0:00:08    31% |##########                      | 164.5MB 12.5MB/s eta 0:00:29.6MB 67.8MB/s eta 0:00:05.4MB 76.2MB/s eta 0:00:05.2MB 65.2MB/s eta 0:00:04    53% |#################               | 277.7MB 62.8MB/s eta 0:00:04.3MB 76.5MB/s eta 0:00:03    91% |#############################   | 471.8MB 70.5MB/s eta 0:00:01 75.7MB/s eta 0:00:01
    100% |################################| 3.9MB 1.4MB/s eta 0:00:01
    100% |################################| 450kB 4.2MB/s eta 0:00:01
    100% |################################| 2.7MB 2.5MB/s eta 0:00:01
    100% |################################| 26.1MB 565kB/s eta 0:00:01
    100% |################################| 61kB 3.4MB/s eta 0:00:01
    1

    100% |################################| 163kB 3.9MB/s eta 0:00:01
    100% |################################| 61kB 3.6MB/s eta 0:00:01
    100% |################################| 143kB 6.0MB/s eta 0:00:01
    100% |################################| 133kB 5.9MB/s eta 0:00:01
    100% |################################| 81kB 3.1MB/s eta 0:00:01
    100% |################################| 153kB 4.0MB/s eta 0:00:01
  Stored in directory: /home/spark/shared/.cache/pip/wheels/b1/c2/ed/d62208260edbd3fa7156545c00ef966f45f2063d0a84f8208a
  Stored in directory: /home/spark/shared/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Stored in directory: /home/spark/shared/.cache/pip/wheels/8e/28/49/fad4e7f0b9a1227708cbbee4487ac8558a7334849cb81c813d
Successfully built wrapt termcolor absl-py
ibm-cos-sdk-core 2.4.3 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.8 which is incompatible.
botocore 1.12.82 has requirement urllib3<1.25,>=1.20, but you'll 

In [1]:
import tensorflow as tf
if not tf.__version__ == '2.2.0-rc0':
    print(tf.__version__)
    raise ValueError('please upgrade to TensorFlow 2.2.0-rc0, or restart your Kernel (Kernel->Restart & Clear Output)')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200328205040-0001
KERNEL_ID = 1ca974b9-d54a-4ede-9f1d-6c83bb6be4e0


IMPORTANT! => Please restart the kernel by clicking on "Kernel"->"Restart and Clear Outout" and wait until all output disapears. Then your changes are beeing picked up

As you can see, we use Keras' Sequential model with only two types of layers: Dense and Dropout. We also specify a random seed to make our results reproducible. Next, we load the Reuters data set:

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
seed = 1337
np.random.seed(seed)
from tensorflow.keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

2113536/2110848 [==============================] - 0s 0us/step


Note that we cap the maximum number of words in a news item to 1000 by specifying the *num_words* key word. Also, 20% of the data will be test data and we ensure reproducibility by setting our random seed.

Our training features are still simply sequences of indexes and we need to further preprocess them, so that we can plug them into a *Dense* layer. For this we use a *Tokenizer* from Keras' text preprocessing module. This tokenizer will take an index sequence and map it to a vector of length *max_words=1000*. Each of the 1000 vector positions corresponds to one of the words in our newswire corpus. The output of the tokenizer has a 1 at the i-th position of the vector, if the word corresponding to i is in the description of the newswire, and 0 otherwise. Even if this word appears multiple times, we still just put a 1 into our vector, i.e. our tokenizer is binary. We use this tokenizer to transform both train and test features:

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

## 1. Exercise part: label encoding

Use to_categorical, as we did in the lectures, to transform both *y_train* and *y_test* into one-hot encoded vectors of length *num_classes*:

In [4]:
y_train = to_categorical(y_train, num_classes=46)
y_test = to_categorical(y_test, num_classes=46)

## 2. Exercise part: model definition

Next, initialise a Keras *Sequential* model and add three layers to it:

    Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
    Layer: Add a *Dropout* layer with dropout rate of 50%.
    Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [5]:
model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu')) # Add first layer. Make sure to specify input shape
model.add(Dropout(0.5)) # Add second layer
model.add(Dense(num_classes, activation='softmax')) # Add third layer

## 3. Exercise part: model compilation

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric. NOTE: In case you get an error regarding h5py, just restart the kernel and start from scratch

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## 4. Exercise part: model training and evaluation

Next, define the batch_size for training as 32 and train the model for 5 epochs on *x_train* and *y_train* by using the *fit* method of your model. Then calculate the score for your trained model by running *evaluate* on *x_test* and *y_test* with the same batch size as used in *fit*.

In [9]:
batch_size = 32
model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=5)
score = model.evaluate(x=x_test, y=y_test)

Epoch 1/5
281/281 [==============================] - 1s 5ms/step - loss: 1.3942 - accuracy: 0.6856
Epoch 2/5
281/281 [==============================] - 2s 5ms/step - loss: 0.7710 - accuracy: 0.8164
Epoch 3/5
281/281 [==============================] - 1s 5ms/step - loss: 0.5554 - accuracy: 0.8653
Epoch 4/5
281/281 [==============================] - 1s 5ms/step - loss: 0.4114 - accuracy: 0.8960
Epoch 5/5
71/71 [==============================] - 0s 2ms/step - loss: 0.8702 - accuracy: 0.7983


If you have done everything as specified, in particular set the random seed as we did above, your test accuracy should be around 80% 

In [10]:
score[1]

0.7983080744743347

Congratulations, now it's time to submit your result to the Coursera grader by executing the following cells (Programming Assingment, Week2). 

We have to install a little library in order to submit to coursera


In [11]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2020-03-28 20:59:31--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: 'rklib.py'

100%[======================================>] 2,540       --.-K/s   in 0s      

2020-03-28 20:59:31 (34.4 MB/s) - 'rklib.py' saved [2540/2540]



Please provide your email address and obtain a submission token (secret) on the grader’s submission page in coursera, then execute the cell

In [12]:
from rklib import submit
import json

key = "XbAMqtjdEeepUgo7OOVwng"
part = "HCvcp"
email = "hhammoud01@gmail.com"###_YOUR_CODE_GOES_HERE_###
token = "vGQatYOoOR9aF9Lh"###_YOUR_CODE_GOES_HERE_### #you can obtain it from the grader page on Coursera (have a look here if you need more information on how to obtain the token https://youtu.be/GcDo0Rwe06U?t=276)


submit(email, token, 'XbAMqtjdEeepUgo7OOVwng', part, [part], json.dumps(score[1]*100))

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"ozVf2","id":"tE4j0qhMEeecqgpT6QjMdA~ozVf2~EYOfjHE3EeqylxK2B1KK5Q","courseId":"tE4j0qhMEeecqgpT6QjMdA"}],"paging":{},"linked":{}}
-------------------------
